In [1]:
import cobra
from cobra.io import load_json_model
import numpy as np
import pandas as pd
import ray
#cobra_model = load_json_model("./COBRA_models/GEM_Recon3_thermocurated_redHUMAN_AA.json")



from cobra.io.mat import *
#from itertools import compress



cobra_model = load_matlab_model("./COBRA_models/GEM_Recon3_thermocurated_redHUMAN_AA.mat")

import itertools

def grow_dict(previo, nuevo):  
    """Esto es un .update(overwritte=False)?"""

    not_new_keys = set(previo.keys()).intersection(set(nuevo.keys()))
    new_keys     = set(nuevo.keys()).difference(set(previo.keys()))
    for new in new_keys:
        previo[new] =np.unique(nuevo[new]).tolist() 
    for common in not_new_keys:
        previo[common].extend(np.unique(nuevo[common]).tolist() )
    return previo
# Sacamos los genes de Recon hacia un diccionario
# TODO: convertir est en una horrible dict_comprehension { curse : cursed for c in curseds }
dict_rxn_to_gene = dict()
genes            = [g.id for g in cobra_model.genes]

for id in genes:
    # fmt: off
    rxns             = [r.id for r in cobra_model.genes.get_by_id(id)._reaction] # reacciones asociadas al gen de la iter
    updated_dict  : dict[str, list[str]] = dict(zip(rxns,itertools.repeat([id]))) # zipping con la { reaccion : [id_gen] }
    dict_rxn_to_gene = grow_dict(dict_rxn_to_gene, updated_dict) # merge recursivo

No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


In [2]:
def get_data_for_heatmap(explanatory_subgraph_path):
        
    explanatory_subgraph = pd.read_parquet(explanatory_subgraph_path).reset_index(drop=True)

    ray.shutdown()
    ray.init()
    @ray.remote


    def get_genes(node_1, node_2):
        
        if node_2 in dict_rxn_to_gene.keys():
            
            return dict_rxn_to_gene[node_2]
            
        elif node_1 in dict_rxn_to_gene.keys():
            
            return dict_rxn_to_gene[node_1]
            
        else:
            return ['']
        
    g = [] 
    for node_1, node_2 in zip(explanatory_subgraph.node1.tolist(), explanatory_subgraph.node2.tolist()):   
        g.append(get_genes.remote(node_1, node_2 ))

    
    genes = pd.Series(ray.get(g) , name='genes')



    assert genes.shape[0] == explanatory_subgraph.shape[0]


    return pd.concat([genes, explanatory_subgraph], axis=1)




to_R_heatmap_explanatory_subgraph_MASKED_GCN_Fluxes = get_data_for_heatmap("./results/dataframes/explanatory_subgraph_MASKED_GCN_Fluxes.parquet.gzip")
to_R_heatmap_explanatory_subgraph_MASKED_GCN_Concentration = get_data_for_heatmap("./results/dataframes/explanatory_subgraph_MASKED_GCN_Concentration.parquet.gzip")
to_R_heatmap_explanatory_subgraph_MASKED_GCN_Concen_plus_Fluxes = get_data_for_heatmap("./results/dataframes/explanatory_subgraph_MASKED_GCN_Concen_plus_Fluxes.parquet.gzip")

2022-12-12 19:01:53,430	INFO worker.py:1518 -- Started a local Ray instance.
2022-12-12 19:02:02,439	INFO worker.py:1518 -- Started a local Ray instance.
2022-12-12 19:02:11,173	INFO worker.py:1518 -- Started a local Ray instance.


In [3]:
to_R_heatmap_explanatory_subgraph_MASKED_GCN_Fluxes.to_csv(
    "./results/dataframes/to_R_heatmap_explanatory_subgraph_MASKED_GCN_Fluxes.csv")
to_R_heatmap_explanatory_subgraph_MASKED_GCN_Concentration.to_csv(
    "./results/dataframes/to_R_heatmap_explanatory_subgraph_MASKED_GCN_Concentration.csv")
to_R_heatmap_explanatory_subgraph_MASKED_GCN_Concen_plus_Fluxes.to_csv(
    "./results/dataframes/to_R_heatmap_explanatory_subgraph_MASKED_GCN_Concen_plus_Fluxes.csv")

# Masked

In [4]:
Masked_edges_node_names_explainer_subgraphs = pd.read_parquet(
    "./results/dataframes/Masked_edges_node_names_explainer_subgraphs.parquet.gzip").reset_index(drop=True)

ray.shutdown()
ray.init()
@ray.remote


def get_genes(node_1, node_2):
    
    if node_2 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_2]
        
    elif node_1 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_1]
        
    else:
        return ['']
    
g = [] 
for node_1, node_2 in zip(Masked_edges_node_names_explainer_subgraphs.node1.tolist(), Masked_edges_node_names_explainer_subgraphs.node2.tolist()):   
    g.append(get_genes.remote(node_1, node_2 ))

 
genes = pd.Series(ray.get(g) , name='genes')



assert genes.shape[0] == Masked_edges_node_names_explainer_subgraphs.shape[0]



Masked_results_for_Heatmaps = pd.concat([genes, Masked_edges_node_names_explainer_subgraphs], axis=1)
Masked_results_for_Heatmaps

2022-12-05 12:25:49,703	INFO worker.py:1518 -- Started a local Ray instance.


,genes,node1,node2,0,1
0,[],TRPGLULEUr,glu_L_c,0.014967,0.014388
1,"[1375.1, 1375.3, 126129.1, 1374.1, 1375.2]",C160CPT1,coa_c,0.015270,0.022956
2,"[1375.1, 1375.3, 126129.1, 1374.1, 1375.2]",C160CPT1,crn_c,0.014483,0.015676
3,[51727.1],CYTK10,cdp_c,0.013181,0.016316
4,[9057.1],r1758,trp_L_c,0.012910,0.016337
...,...,...,...,...,...
500,[160728.1],L_LACt4r,lac_L_e,0.012621,0.017381
501,[51727.1],CYTK5,dcmp_c,0.012556,0.017218
502,[],AHCYStr,ahcys_r,0.012489,0.017030
503,"[6530.1, 6531.1]",DOPAt4_2_r,dopa_c,0.012889,0.016586


In [5]:
Masked_results_for_Heatmaps.to_csv(
    "./results/dataframes/Masked_results_for_Heatmaps.csv")